# Real time Object Detection

O Arquivo está dividido em vários pedaços. 
Fazer o download do arquivo .tar.gz e executar o passos: 

Juntar novamente o arquivo 
cat RealTimeObjectDetection-* > RealTimeObjectDetection.tar.gz 

Descompactar 
tar xvzf RealTimeObjectDetection.tar.gz

Executar o Jupyter Notebook (arquivo .ipynb) 

============================================================================================

O arquivo foi criado como os seguintes comandos: 

Compactar um diretório/folder 
tar -czvf RealTimeObjectDetection.tar.gz RealTimeObjectDetection

Separar o arquivo em pedaços de 20MB 
split -b 20000000 RealTimeObjectDetection.tar.gz RealTimeObjectDetection-

In [1]:
import os
import cv2
import time
import argparse
import multiprocessing
import numpy as np
import tensorflow as tf

from utils.app_utils import FPS, WebcamVideoStream
from multiprocessing import Queue, Pool
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

CWD_PATH = os.getcwd()

# Este é o modelo real que é usado para a detecção de objetos.
MODEL_NAME = 'ssd_mobilenet_v1_coco_11_06_2017'
PATH_TO_CKPT = os.path.join(CWD_PATH, 'object_detection', MODEL_NAME, 'frozen_inference_graph.pb')

# Lista das strings que são usadas para adicionar o rótulo correto para cada caixa.
PATH_TO_LABELS = os.path.join(CWD_PATH, 'object_detection', 'data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

# Carregando mapa de labels
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, 
                                                            use_display_name=True)
category_index = label_map_util.create_category_index(categories)


def detect_objects(image_np, sess, detection_graph):
    # Expande as dimensões uma vez que o modelo espera que as imagens tenham shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

    # Cada caixa representa uma parte da imagem onde um objeto específico foi detectado.
    boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

    # Cada score representa o nível de confiança para cada um dos objetos. 
    # O score é mostrado na imagem do resultado, juntamente com o rótulo da classe.
    scores = detection_graph.get_tensor_by_name('detection_scores:0')
    classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')

    # Detecção
    (boxes, scores, classes, num_detections) = sess.run([boxes, scores, classes, num_detections], 
                                                        feed_dict={image_tensor: image_np_expanded})

    # Visualização dos resultados de uma detecção.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        np.squeeze(boxes),
        np.squeeze(classes).astype(np.int32),
        np.squeeze(scores),
        category_index,
        use_normalized_coordinates=True,
        line_thickness=8)
    return image_np


def worker(input_q, output_q):
    # Carrega o modelo TensorFlow na memória.
    detection_graph = tf.Graph()
    with detection_graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')

        sess = tf.Session(graph=detection_graph)

    fps = FPS().start()
    while True:
        fps.update()
        frame = input_q.get()
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        output_q.put(detect_objects(frame_rgb, sess, detection_graph))

    fps.stop()
    sess.close()


if __name__ == '__main__':
    """   
    parser = argparse.ArgumentParser()
    parser.add_argument('-src', '--source', dest='video_source', type=int, default=0,
                        help='Device index of the camera.')
    parser.add_argument('-wd', '--width', dest='width', type=int, default=480, 
                        help='Width of the frames in the video stream.')
    parser.add_argument('-ht', '--height', dest='height', type=int, default=360, 
                        help='Height of the frames in the video stream.')
    parser.add_argument('-num-w', '--num-workers', dest='num_workers', type=int, default=2, 
                        help='Number of workers.')
    parser.add_argument('-q-size', '--queue-size', dest='queue_size', type=int, default=5, help='Size of the queue.')
    args = parser.parse_args()
    """

    logger = multiprocessing.log_to_stderr()
    logger.setLevel(multiprocessing.SUBDEBUG)

    input_q = Queue(maxsize=5)
    output_q = Queue(maxsize=5)
    pool = Pool(2, worker, (input_q, output_q))

    video_capture = WebcamVideoStream(src=0, width=480, height=360).start()
    fps = FPS().start()

    while True:  
        frame = video_capture.read()
        input_q.put(frame)

        t = time.time()

        output_rgb = cv2.cvtColor(output_q.get(), cv2.COLOR_RGB2BGR)
        cv2.imshow('Video', output_rgb)
        fps.update()

        print('[INFO] Tempo: {:.2f}'.format(time.time() - t))

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    fps.stop()
    print('[INFO] Tempo Total: {:.2f}'.format(fps.elapsed()))
    print('[INFO] FPS - Frames Por Segundos (aproximado): {:.2f}'.format(fps.fps()))

    pool.terminate()
    video_capture.stop()
    cv2.destroyAllWindows()

/Users/vladimiralencar/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)
[DEBUG/MainProcess] created semlock with handle 61
[DEBUG/MainProcess] created semlock with handle 62
[DEBUG/MainProcess] created semlock with handle 63
[DEBUG/MainProcess] Queue._after_fork()
[DEBUG/MainProcess] created semlock with handle 66
[DEBUG/MainProcess] created semlock with handle 67
[DEBUG/MainProcess] created semlock with handle 68
[DEBUG/MainProcess] Queue._after_fork()
[DEBUG/MainProcess] created semlock with handle 71
[DEBUG/MainProcess] created semlock with handle 72
[DEBUG/MainProcess] created semlock with handle 75
[DEBUG/MainProcess] created semlock with handle 76
[DEBUG/MainProcess] added worker
[DEBUG/MainProcess] added worker
[DEBUG/ForkPoolWorker-1] Queue._after_fork()
[DEBUG/ForkPoolWorker-1] Queue._after_fork()
[DEBUG/ForkPoolWorke

[INFO] Tempo: 9.36


[DEBUG/ForkPoolWorker-1] Queue._start_thread()
[DEBUG/ForkPoolWorker-1] doing self._thread.start()
[DEBUG/ForkPoolWorker-1] starting thread to feed data to pipe
[DEBUG/ForkPoolWorker-1] ... done self._thread.start()


[INFO] Tempo: 4.41
[INFO] Tempo: 0.21
[INFO] Tempo: 0.20
[INFO] Tempo: 0.23
[INFO] Tempo: 0.28
[INFO] Tempo: 0.26
[INFO] Tempo: 0.31
[INFO] Tempo: 0.26
[INFO] Tempo: 0.20
[INFO] Tempo: 0.26
[INFO] Tempo: 0.26
[INFO] Tempo: 0.25
[INFO] Tempo: 0.22
[INFO] Tempo: 0.21
[INFO] Tempo: 0.27
[INFO] Tempo: 0.26
[INFO] Tempo: 0.27
[INFO] Tempo: 0.26
[INFO] Tempo: 0.24
[INFO] Tempo: 0.19
[INFO] Tempo: 0.21
[INFO] Tempo: 0.22
[INFO] Tempo: 0.27
[INFO] Tempo: 0.23
[INFO] Tempo: 0.28
[INFO] Tempo: 0.25
[INFO] Tempo: 0.23
[INFO] Tempo: 0.24
[INFO] Tempo: 0.23
[INFO] Tempo: 0.23


[DEBUG/MainProcess] terminating pool
[Level 5/MainProcess] finalizer calling <bound method Pool._terminate_pool of <class 'multiprocessing.pool.Pool'>> with args (<_queue.SimpleQueue object at 0xb20437548>, <multiprocessing.queues.SimpleQueue object at 0xb20570b70>, <multiprocessing.queues.SimpleQueue object at 0xb20570fd0>, [<ForkProcess(ForkPoolWorker-1, started daemon)>, <ForkProcess(ForkPoolWorker-2, started daemon)>], <Thread(Thread-4, started daemon 123145528209408)>, <Thread(Thread-5, started daemon 123145533464576)>, <Thread(Thread-6, started daemon 123145538719744)>, {}) and kwargs {}
[DEBUG/MainProcess] finalizing pool
[DEBUG/MainProcess] helping task handler/workers to finish
[DEBUG/MainProcess] removing tasks from inqueue until task handler finished
[DEBUG/MainProcess] result handler found thread._state=TERMINATE
[DEBUG/MainProcess] joining worker handler
[DEBUG/MainProcess] ensuring that outqueue is not full
[DEBUG/MainProcess] result handler exiting: len(cache)=0, thread.

[INFO] Tempo: 0.23
[INFO] Tempo Total: 21.86
[INFO] FPS - Frames Por Segundos (aproximado): 1.46


[DEBUG/MainProcess] cleaning up worker 51200
